In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, logarithmic
from rtbm import minimizer

from rtbm.activations import tanh, linear, sigmoid
from rtbm.mathtools import theta_1d, logtheta_1d_phaseI
from rtbm.riemann_theta.riemann_theta import RiemannTheta
from rtbm.gradientschemes import RMSprop, adam, adadelta, adagrad

from rtbm.initializers import uniform

import rtbm.layers as layers
import rtbm.model as mdl

from rtbm import RTBM

from numpy import frompyfunc

## NormAddLayer grad test

In [2]:
M = mdl.Model()
M.add(layers.NormAddLayer(2,1))

In [3]:
n=10
data = (np.random.uniform(0,1,n))
data=np.append(data,1-data).reshape(2,n)
print(data)

W=M.get_layer(1)._w
print("W:",W)
print("Ws:",np.sum(W, axis=1))
R = M.feed_through(data)
print(R)

[[0.35586492 0.71422209 0.32320333 0.7298183  0.23908343 0.99253002
  0.62902895 0.1951772  0.12111821 0.4759044 ]
 [0.64413508 0.28577791 0.67679667 0.2701817  0.76091657 0.00746998
  0.37097105 0.8048228  0.87888179 0.5240956 ]]
('W:', array([[-0.57103509,  1.22549187]]))
('Ws:', array([0.65445678]))
[[0.60312162 0.34673453 0.62648941 0.3355762  0.68667309 0.14761877
  0.40768607 0.71808586 0.77107146 0.51723923]]


In [4]:
np.exp(M.get_layer(1)._w).dot(data)/np.sum(np.exp(W), axis=1)

array([[0.60312162, 0.34673453, 0.62648941, 0.3355762 , 0.68667309,
        0.14761877, 0.40768607, 0.71808586, 0.77107146, 0.51723923]])

In [8]:
M.gradient_check(1,data[:,0:10].reshape((2,10)),0.0001)

('I: ', array([[0.35586492, 0.71422209, 0.32320333, 0.7298183 , 0.23908343,
        0.99253002, 0.62902895, 0.1951772 , 0.12111821, 0.4759044 ],
       [0.64413508, 0.28577791, 0.67679667, 0.2701817 , 0.76091657,
        0.00746998, 0.37097105, 0.8048228 , 0.87888179, 0.5240956 ]]))
('P: ', array([-0.57103509,  1.22549187]))
('O: ', array([[0.60312162, 0.34673453, 0.62648941, 0.3355762 , 0.68667309,
        0.14761877, 0.40768607, 0.71808586, 0.77107146, 0.51723923]]))
1 th (mean) numerical gradient: 
[0.00546825]
1 th (mean) backprop gradient: 
0.00546824964222325


## RTBM grad descent test

In [ ]:
# simple gaussian
n = 1000
data = (np.random.normal(5,10,n)).reshape(1,n)
plt.hist(data.T, bins=50, normed=True);

M = mdl.Model()
M.add(layers.ThetaUnitLayer(1,1,1,diagonal_T=True))



In [ ]:
# Flow to a non-trivial W
minim = minimizer.SGD()
solution = minim.train(logarithmic, M, data,scheme=adagrad(), lr=0.000001, maxiter=500)

In [ ]:
print(M.get_layer(1).get_unit(1)._q)
print(M.get_layer(1).get_unit(1)._w)
#M.get_layer(1).get_unit(1)._bv[0,0] = 0

print(M.get_layer(1).get_unit(1)._bv)

M.get_parameters()
M.gradient_check(3,data[:,0:10].reshape((1,10)),0.0001)


### Derivatives test

In [ ]:
Q=np.zeros((1,1), dtype=complex)
V=np.zeros((1,1), dtype=complex)
V[0,0]=0.1*2j*np.pi
Q[0,0]=0.01*2*np.pi

#print(theta_1d(V,Q,0))


print(RiemannTheta(V/(2j*np.pi),1j*Q/(2*np.pi), derivs=[[1],[1]]))

#print(RiemannTheta(V/(2j*np.pi),1j*Q/(2*np.pi), derivs=[[1],[1]]))

#mpmath.jtheta(3,V[0,0],Q[0,0])

In [ ]:
X=np.random.uniform(-50, 50,(1000,1))

In [ ]:
%%time
t = RiemannTheta(X/(2j*np.pi),1j*Q/(2*np.pi))

In [ ]:
Q=np.zeros((1,1), dtype=float)
V=np.zeros((1,1), dtype=float)
V[0,0]=0.1
Q[0,0]=0.01


print(theta_1d(V,Q,0))
print(np.exp(logtheta_1d_phaseI(V,Q,0)))

print(RiemannTheta(V/(2j*np.pi),-1*Q/(2j*np.pi)))

# Linear layer test

In [ ]:
def funcA(x):
    return 0.6-0.3*x

def funcB(x):
    return -0.5+0.8*x


def func(x1,x2):
    return 0.6-0.3*x1+1.2*x2

X1 = np.linspace(-5.3, 5, 997)
X2 = np.linspace(-5.5, 5, 997)

X = np.stack((X1,X2))

#Y = func(X1,X2).reshape((1,X.shape[1]))
Y = np.stack((funcA(X1),funcB(X2)))

M = mdl.Model()
M.add(layers.Linear(2,2))

#minim = minimizer.CMA(True)
#minim.train(mse(), M, X, Y, tolfun=1e-3)

M = mdl.Model()
M.add(layers.Linear(2,1))
M.add(layers.Linear(1,2))


minim = minimizer.SGD()
minim.train(mse(), M, X, Y, maxiter=300,batch_size=98)

# E(h|v) SGD test

## Phase I

In [ ]:
def func(x):
    return np.sin(x)+x

X = np.linspace(-3.5, 5, 998)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,3, phase=1j, Q_init=uniform(2,3+1e-5)))
M.add(layers.DiagExpectationUnitLayer(3,1, phase=1j, Q_init=uniform(2,3+1e-5)))

#M.add(layers.DiagExpectationUnitLayer(3,1, phase=1))


In [ ]:

#print(M.get_parameters())
M.gradient_check(13,X,0.01)

In [ ]:
minim = minimizer.SGD()

minim.train(mse(), M, X, Y, lr=0.01,maxiter=1000)

plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

## Phase II

In [ ]:
def func(x):
    return np.sin(x)

X = np.linspace(-5, 5, 997)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3, phase=1j, Q_init=uniform(2,3+1e-5)))
M.add(layers.DiagExpectationUnitLayer(3,1, phase=1j, Q_init=uniform(2,3+1e-5)))

print("*** init ***")
print(M.get_layer(1)._q)
print(M.get_layer(2)._q)

minim = minimizer.SGD()

minim.train(mse(), M, X, Y, lr=0.1, scheme=RMSprop(), maxiter=500)

plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

In [ ]:
print(M.get_layer(1)._q)
print(M.get_layer(2)._q)


## CMA

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3,phase=1))
M.add(layers.DiagExpectationUnitLayer(3,1,phase=1))

minim = minimizer.CMA()

minim.train(mse(), M, X, Y, maxiter=500)
plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

In [ ]:
def func(x1,x2):
    return 0.2*x1+0.4*x2+0.8

X1 = np.linspace(-5.3, 5, 1000)
X2 = np.linspace(-5.5, 5, 1000)

X = np.stack((X1,X2))

Y = func(X1[:,None],X2[:,None]).reshape((1,X1.shape[0]))

M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(2,1))
M.add(layers.DiagExpectationUnitLayer(1,1))


minim = minimizer.SGD()
minim.train(mse(), M, X, Y, lr=0.1,maxiter=400)

#plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
#plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

# Layer test

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3))
M.add(layers.DiagExpectationUnitLayer(3,1))

In [ ]:
def func(x):
    return np.sin(x)

X = np.linspace(0, 10, 5)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
minim = minimizer.CMA(True)
minim.train(mse(), M, X, Y, tolfun=1e-3)

In [ ]:
npoints = 5
test_X = (np.linspace(0, 10, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Misc tests

In [ ]:
L = layers.MaxPosLayer(3,1)

In [ ]:
L.feedin(np.array([Y,2*Y,1*Y]).reshape(3,5))

In [ ]:
np.array([Y,2*Y,Y]).reshape(3,5)

In [ ]:
np.empty(0)

In [ ]:
derivative_1d_theta_phaseI([0.1],[1j],0)